# Running Containers

## Verify docker service is running

In [1]:
systemctl status docker.service

## Confirm docker version

In [2]:
docker version

## Search for an image on the docker hub

In [3]:
docker search mssql

## Get tags from Microsoft Container Registry

In [4]:
$repo = invoke-webrequest https://mcr.microsoft.com/v2/mssql/server/tags/list -UseBasicParsing
$repo.content

## Pull image down locally

In [5]:
docker image pull mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Check local images

In [6]:
docker image ls

## Run a container

In [7]:
docker container run -d -p 15111:1433 `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--name testcontainer1 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

# Verify container is running

In [8]:
docker container ls -a

## Check container logs

In [9]:
docker container logs testcontainer1

## Connect to SQL instance and confirm version

In [10]:
mssql-cli -S 'localhost,15111' -U sa -P Testing1122 -Q "SELECT @@VERSION;"

In [11]:
# let's have a look within the container
docker container exec testcontainer1 bash -c "ls /var/opt/mssql/data/"

## Copy a backup file into the container

In [12]:
docker container cp ~/git/SQLServerAndContainers/DatabaseBackup/DatabaseA.bak `
        testcontainer1:/var/opt/mssql/data/

## Check that the backup file has been copied into the container

In [13]:
docker container exec testcontainer1 bash -c "ls /var/opt/mssql/data"

## Restore the database in the container

In [14]:
mssql-cli -S 'localhost,15111' -U sa -P Testing1122 `
-Q "RESTORE DATABASE [DatabaseA] FROM DISK = '/var/opt/mssql/data/DatabaseA.bak';"

## Check that the database has been restored

In [15]:
mssql-cli -S 'localhost,15111' -U sa -P Testing1122 `
-Q "select name from sys.databases;"

## Run a couple more of containers

In [16]:
docker container run -d -p 15222:1433 `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--name testcontainer2 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

docker container run -d -p 15333:1433 `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--name testcontainer3 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Verify containers are running

In [17]:
docker container ls -a

## View stats on container usage

In [18]:
docker container stats --no-stream

## Run a container with resources limited

In [20]:
docker container run -d -p 15444:1433 `
--cpus=2 --memory=2048m `
--env ACCEPT_EULA=Y --env SA_PASSWORD=Testing1122 `
--name testcontainer4 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Check container is running

In [21]:
docker container ls -a

## Check the stats

In [22]:
docker container stats --no-stream

## Clean up

In [23]:
docker container rm $(docker ps -a -q) -f